## Topographic Complexity/Variability: Rugosity  
Developed in November 2023 by Dr. Larry Syu-Heng Lai (University of Washington)  

Recommended reference:
* Wilson, M.F.J., O’Connell, B., Brown, C., Guinan, J.C., Grehan, A.J., 2007. Multiscale Terrain Analysis of Multibeam Bathymetry Data for Habitat Mapping on the Continental Slope. Marine Geodesy 30, 3-35. https://doi.org/10.1080/01490410701295962 
* Du Preez, C. A new arc–chord ratio (ACR) rugosity index for quantifying three-dimensional landscape structural complexity. Landscape Ecol 30, 181–192 (2015). https://doi.org/10.1007/s10980-014-0118-8

### Initial setup

In [42]:
import numpy as np
import rasterio
import joblib
from scipy.linalg import lstsq
from joblib import Parallel, delayed
from tqdm.notebook import tqdm

### Define data path

In [43]:
# Define your file paths and file names separately
input_folder = '/Users/larryslai/Library/CloudStorage/Dropbox/QGIS/WA LiDAR/'
input_file_name = 'Test_DEM.tif'
#input_file_name = 'Tokeland_DEM.tif'
#input_file_name = 'Nemah_DEM.tif'
#input_file_name = 'Francies_DEM.tif'

output_folder = '/Users/larryslai/Library/CloudStorage/Dropbox/QGIS/WA LiDAR/'
output_file_name = 'Test_pyRugosity.tif'
#output_file_name = 'Tokeland_pyRugosity.tif'
#output_file_name = 'Nemah_pyRugosity.tif'
#output_file_name = 'Francies_pyRugosity.tif'

# Combine folder and file names to create the full paths
input_tif_path = input_folder + input_file_name
output_tif_path = output_folder + output_file_name

### Read a DEM

In [44]:
with rasterio.open(input_tif_path) as src:
    dem = src.read(1)  # Read the first band into a 2D array
    meta = src.meta

See coordinate system info of the GeoTIFF

In [45]:
# Open the GeoTIFF file
with rasterio.open(input_tif_path) as src:
    # Read the CRS
    crs = src.crs
    
    # Print the CRS information
    print(f"CRS: {crs}")
    print(f"CRS as WKT: {crs.wkt}")
    print(f"CRS as PROJ string: {crs.to_proj4()}")
    print(f"CRS as EPSG code: {crs.to_epsg()}")
    print(f"CRS as dictionary: {crs.to_dict()}")

CRS: PROJCS["NAD83(HARN) / Washington South (ftUS)",GEOGCS["NAD83(HARN)",DATUM["North_American_1983_HARN",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",45.3333333333333],PARAMETER["central_meridian",-120.5],PARAMETER["standard_parallel_1",45.8333333333333],PARAMETER["standard_parallel_2",47.3333333333333],PARAMETER["false_easting",1640416.66666667],PARAMETER["false_northing",0],UNIT["US survey foot",0.304800609601219,AUTHORITY["EPSG","9003"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
CRS as WKT: PROJCS["NAD83(HARN) / Washington South (ftUS)",GEOGCS["NAD83(HARN)",DATUM["North_American_1983_HARN",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitud

## Rugosity (slope-corrected)

Rugosity measures the complexity of the surface texture and is defined as the ratio of the actual surface area to the planar area. It is calculated using a $N \times N$ neighborhood around each pixel:

$$
\text{Rugosity Index} = \frac{\text{surface area of NxN neighborhood}}{\text{planar area of NxN neighborhood}}
$$

, where $N$ is the window size. In practice, the surface area is estimated using the gradients of the elevation within the neighborhood, accounting for the additional area contributed by the terrain's slope.  

The planer area should be the area of the surface orthogonally projected onto a plane of best fit within the window. To calculate the planar area as the area of the surface orthogonally projected onto a plane of best fit within the window, we need to compute the local slope and aspect for each cell within the window, and then use these to project the area onto the best-fit plane. After this local slope correction, the result is the indext so-called arc-chord ratio (ACR) rugosity index. See details in Du Preez (2015).

##### Rugosity function  

In [46]:
def fit_plane_to_window(window):
    """
    Fit a plane to a window of elevation data.

    :param window: 2D array of elevation values.
    :return: Coefficients of the plane.
    """
    window_size = window.shape[0]
    x, y = np.indices(window.shape)
    A = np.c_[x.ravel(), y.ravel(), np.ones(window.size)]
    C, _, _, _ = lstsq(A, window.ravel())  # Coefficients of the plane
    return C

In [47]:
def calculate_orthogonal_projected_area(window, coeffs):
    """
    Calculate the area of the surface orthogonally projected onto a plane of best fit.

    :param window: 2D array of elevation values.
    :param coeffs: Coefficients of the plane.
    :return: Projected area of the surface.
    """
    # Calculate normal vector to the plane
    nx, ny, nz = coeffs[0], coeffs[1], -1
    normal = np.array([nx, ny, nz])
    normal_length = np.linalg.norm(normal)
    
    # Calculate area of the projected plane
    window_size = window.shape[0]
    projected_area = window_size**2 / normal_length
    return projected_area

In [48]:
def calculate_rugosity_for_window(dem, i, j, window_size):
    """
    Calculate rugosity for a single window in the raster.
    
    :param dem: 2D array of elevation values.
    :param i: Row index for the center of the window.
    :param j: Column index for the center of the window.
    :param window_size: Size of the moving window to calculate the rugosity.
    :return: Rugosity value for the window.
    """
    window = dem[i:i+window_size, j:j+window_size]
    coeffs = fit_plane_to_window(window)
    projected_area = calculate_orthogonal_projected_area(window, coeffs)
    surface_area = np.sum(np.sqrt(1 + np.gradient(window)[0]**2 + np.gradient(window)[1]**2))
    return surface_area / projected_area

In [49]:
def calculate_rugosity(dem, window_size):
    """
    Calculate rugosity for each cell in the raster using parallel processing.
    
    :param dem: 2D array of elevation values.
    :param window_size: Size of the moving window to calculate the rugosity.
    :return: 2D array of rugosity values.
    """
    rows, cols = dem.shape
    rugosity_map = np.zeros_like(dem, dtype=np.float32)
    half_window = window_size // 2
    
    # Prepare indices for parallel processing
    indices = [(i, j) for i in range(half_window, rows - half_window)
                      for j in range(half_window, cols - half_window)]

    # Define the number of jobs for parallel processing
    num_cores = joblib.cpu_count()

    # Parallel processing
    results = Parallel(n_jobs=num_cores)(delayed(calculate_rugosity_for_window)(dem, i, j, window_size) for i, j in tqdm(indices, desc='Calculating Rugosity'))

    # Fill the rugosity map with the results
    for ((i, j), rugosity) in zip(indices, results):
        rugosity_map[i, j] = rugosity

    return rugosity_map

Calculate Rugosity with a given window size

In [50]:
# Calculate rugosity for the entire DEM with a specified window size
window_size = 3  # N x N neighborhood
rugosity_map = calculate_rugosity(dem, window_size)

Calculating Rugosity:   0%|          | 0/1849766 [00:00<?, ?it/s]

Output data into GeoTIFF
* Enabling geotiff compression to reduce writing time
* Enabling Tile-based writing if needed
* Enabling BIGTIFF parameter to allow writing a large GeoTIFF

In [51]:
# Update metadata for output GeoTIFF
meta.update(dtype=rasterio.float32, compress='lzw', tiled=True, bigtiff='IF_SAFER')

# Write Rugosity to a new GeoTIFF
with rasterio.open(output_tif_path, 'w', **meta) as dst:
   dst.write(rugosity_map.astype(rasterio.float32), 1)